
# NCars + GraphRes (AEGNN-style) — Load & Forward Sanity Check

This notebook verifies:
- Loading **AEGNN-style** NCars graphs from `ProcessNCars1.NCarsEventsGraphDataset1`
- Creating PyG DataLoaders
- Running a **GraphRes** forward pass (uses `SplineConv` and expects `edge_attr=Cartesian`)

> If `GraphRes` or `aegnn` package isn't importable here, a **minimal fallback** model will be defined so you can still test loading and a forward pass.


In [1]:

# --- Configuration ---
ROOT = r"C:\Users\hanne\Documents\Hannes\Uni\Maastricht\Project\GNNBenchmark\src\Models\AEGNN\data\ncars"
RADIUS = 3.0
D_MAX  = 32
N_SAMPLES = 10000
SAMPLING  = True
LIMIT_PER_SPLIT = 20

import torch
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE


device(type='cpu')

In [2]:

# Ensure we can import your uploaded ProcessNCars1.py from /mnt/data
import sys
from pathlib import Path
if str(Path('/mnt/data')) not in sys.path:
    sys.path.append(str(Path('/mnt/data')))

from torch_geometric.loader import DataLoader

try:
    from ProcessNCars1 import NCarsEventsGraphDataset1
    print("✅ Imported NCarsEventsGraphDataset1 from ProcessNCars1.py")
except Exception as e:
    print("❌ Could not import NCarsEventsGraphDataset1:", repr(e))
    raise


✅ Imported NCarsEventsGraphDataset1 from ProcessNCars1.py


In [3]:

from pathlib import Path
root = Path(ROOT)
train_dir = root / "training"
val_dir   = root / "validation"
test_dir  = root / "test"
for d in [train_dir, val_dir, test_dir]:
    assert d.exists(), f"Missing split directory: {d}"

train_ds = NCarsEventsGraphDataset1(str(train_dir), r=RADIUS, d_max=D_MAX,
                                    n_samples=N_SAMPLES, sampling=SAMPLING, cache=False)
val_ds   = NCarsEventsGraphDataset1(str(val_dir),   r=RADIUS, d_max=D_MAX,
                                    n_samples=N_SAMPLES, sampling=SAMPLING, cache=False)
test_ds  = NCarsEventsGraphDataset1(str(test_dir),  r=RADIUS, d_max=D_MAX,
                                    n_samples=N_SAMPLES, sampling=SAMPLING, cache=False)

from torch.utils.data import Subset
def take_first(ds, n):
    return Subset(ds, list(range(min(n, len(ds))))) if n and len(ds) > n else ds

if LIMIT_PER_SPLIT and LIMIT_PER_SPLIT > 0:
    train_ds = take_first(train_ds, LIMIT_PER_SPLIT)
    val_ds   = take_first(val_ds, LIMIT_PER_SPLIT)
    test_ds  = take_first(test_ds, LIMIT_PER_SPLIT)

print(f"Datasets ready: train={len(train_ds)} val={len(val_ds)} test={len(test_ds)}")

# Peek a few samples
for i in range(min(3, len(train_ds))):
    d = train_ds[i]
    print(f"[train {i}] x={tuple(d.x.shape)} pos={tuple(d.pos.shape)} ei={tuple(d.edge_index.shape)} "
          f"ea={tuple(d.edge_attr.shape)} y={d.y.item()}")


Datasets ready: train=20 val=20 test=20
[train 0] x=(6263, 1) pos=(6263, 3) ei=(2, 190656) ea=(190656, 3) y=1
[train 1] x=(3250, 1) pos=(3250, 3) ei=(2, 66166) ea=(66166, 3) y=1
[train 2] x=(582, 1) pos=(582, 3) ei=(2, 3757) ea=(3757, 3) y=1


In [4]:

train_loader = DataLoader(train_ds, batch_size=8, shuffle=True,  num_workers=0)
val_loader   = DataLoader(val_ds,   batch_size=8, shuffle=False, num_workers=0)
test_loader  = DataLoader(test_ds,  batch_size=1, shuffle=False, num_workers=0)

batch = next(iter(train_loader))
print("Batch:")
print("  x   :", tuple(batch.x.shape))
print("  pos :", tuple(batch.pos.shape))
print("  ei  :", tuple(batch.edge_index.shape))
print("  ea  :", tuple(batch.edge_attr.shape))
print("  y   :", tuple(batch.y.shape))
print("  batch vector:", tuple(batch.batch.shape))


Batch:
  x   : (12663, 1)
  pos : (12663, 3)
  ei  : (2, 280293)
  ea  : (280293, 3)
  y   : (8,)
  batch vector: (12663,)


In [5]:

import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import SplineConv, global_mean_pool

GRAPHRES_IMPORTED = False
try:
    from GraphRes import GraphRes
    GRAPHRES_IMPORTED = True
    print("✅ Imported GraphRes from your project")
except Exception as e:
    print("⚠️  Could not import GraphRes, using minimal fallback. Reason:", repr(e))

class MinimalGraphRes(nn.Module):
    def __init__(self, hidden=64, num_classes=2):
        super().__init__()
        self.conv1 = SplineConv(1, hidden, dim=3, kernel_size=5, aggr='mean')
        self.bn1   = nn.BatchNorm1d(hidden)
        self.conv2 = SplineConv(hidden, hidden, dim=3, kernel_size=5, aggr='mean')
        self.bn2   = nn.BatchNorm1d(hidden)
        self.fc    = nn.Linear(hidden, num_classes)

    def forward(self, data):
        x = F.elu(self.conv1(data.x, data.edge_index, data.edge_attr)); x = self.bn1(x)
        x = F.elu(self.conv2(x, data.edge_index, data.edge_attr));      x = self.bn2(x)
        batch = getattr(data, "batch", None)
        if batch is None:
            batch = torch.zeros(x.size(0), dtype=torch.long, device=x.device)
        z = global_mean_pool(x, batch)
        return self.fc(z)

if GRAPHRES_IMPORTED:
    import torch
    input_shape = torch.tensor([120, 100, 3])
    model = GraphRes(dataset="ncars", input_shape=input_shape, num_outputs=2,
                     pooling_size=(16,12), bias=False, root_weight=False).to(DEVICE)
else:
    model = MinimalGraphRes(hidden=64, num_classes=2).to(DEVICE)

print(model.__class__.__name__, "on", DEVICE)


✅ Imported GraphRes from your project
GraphRes on cpu


In [6]:

model.eval()
b = next(iter(train_loader)).to(DEVICE)
with torch.no_grad():
    out = model(b)
print("Logits shape:", tuple(out.shape))


C:\Users\hanne\anaconda3\envs\AEGNN\lib\site-packages\torch_geometric\nn\conv\spline_conv.py:133: UserWarning: We do not recommend using the non-optimized CPU version of `SplineConv`. If possible, please move your data to GPU.
  warnings.warn(


Logits shape: (8, 2)


In [7]:

crit = nn.CrossEntropyLoss()
opt  = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=0)

def run_epoch(loader, train=False):
    (model.train() if train else model.eval())
    tot, correct, loss_sum = 0, 0, 0.0
    for batch in loader:
        batch = batch.to(DEVICE)
        out = model(batch)
        loss = crit(out, batch.y.view(-1))
        if train:
            opt.zero_grad(); loss.backward(); opt.step()
        loss_sum += float(loss.item()) * batch.num_graphs
        tot += batch.num_graphs
        correct += int((out.argmax(-1) == batch.y.view(-1)).sum())
    return loss_sum/max(tot,1), correct/max(tot,1)

tr_loss, tr_acc = run_epoch(train_loader, train=True)
va_loss, va_acc = run_epoch(val_loader,   train=False)
print(f"Epoch 1 | train {tr_loss:.4f}/{tr_acc:.3f} | val {va_loss:.4f}/{va_acc:.3f}")


Epoch 1 | train 1.6721/0.350 | val 0.6937/0.500


In [8]:

USE_ASYNC = True
if USE_ASYNC:
    try:
        from torch_geometric.transforms import Cartesian
        from aegnn.asyncronous import make_model_asynchronous
        edge_attr_tf = Cartesian(cat=False, max_value=10.0)
        model = make_model_asynchronous(model, RADIUS, [120, 100], edge_attr_tf).to(DEVICE)
        print("✅ AEGNN async enabled")
    except Exception as e:
        print("ℹ️  Async wrap skipped or failed (okay for this check):", repr(e))

model.eval()
with torch.no_grad():
    out = model(next(iter(test_loader)).to(DEVICE))
print("Post-async/test forward OK, logits:", tuple(out.shape))


✅ AEGNN async enabled
Post-async/test forward OK, logits: (1, 2)


In [ ]:
from evaluate_flops import evaluate_async_aegnn
te_loss, te_acc = evaluate_async_aegnn(
    model, test_ds, "cpu",
    r=3.0, dims=(120,100),
    init_events=1000,
    stream_events=1,
    d_max=32,
    criterion=crit,
    max_sequences=2,       # <- only evaluate 2 sequences
    wrap_if_needed=True
)
